In [1]:
import numpy as np
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from data.base_data_module import BaseDataset

C:\Users\Filipe\.conda\envs\wav2kp\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [2]:
# train_dataset = np.load("dataset/Train.npy", allow_pickle=True)
# test_dataset = np.load("dataset/Test.npy", allow_pickle=True)
# val_dataset = np.load("dataset/Val.npy", allow_pickle=True)

train_keypoints = np.load("dataset/Train_keypoints.npy",  allow_pickle=True)
test_keypoints = np.load("dataset/Test_keypoints.npy",  allow_pickle=True)
val_keypoints = np.load("dataset/Val_keypoints.npy",  allow_pickle=True)

train_mfccs = torch.load("dataset/Train_mfccs.pt" )
test_mfccs = torch.load("dataset/Test_mfccs.pt" )
val_mfccs = torch.load("dataset/Val_mfccs.pt" )

train_mfccs_not_transp = torch.load("dataset/Train_mfccs.pt" )
train_mfccs_not_transp = torch.transpose(train_mfccs_not_transp, 2, 1)

train_set  = BaseDataset(data=train_mfccs, targets=train_keypoints)

print(train_mfccs_not_transp.shape)
train_set  = BaseDataset(data=train_mfccs_not_transp, targets=train_keypoints)


train_loader = DataLoader(train_set)
# test_loader = DataLoader(test_dataset)
# val_loader = DataLoader(val_dataset)

torch.Size([80, 1600, 721])


In [3]:


train_keypoints = np.load("dataset/Train_keypoints.npy",  allow_pickle=True)


train_mfccs = torch.load("dataset/Train_mfccs.pt" )


train_mfccs_not_transp = torch.load("dataset/Train_mfccs.pt" )
train_mfccs_not_transp = torch.transpose(train_mfccs_not_transp, 2, 1)

train_set  = BaseDataset(data=train_mfccs, targets=train_keypoints)
train_set_nt  = BaseDataset(data=train_mfccs_not_transp, targets=train_keypoints)

train_loader = DataLoader(train_set)
train_loader_nt = DataLoader(train_set_nt)


In [4]:
lstm_input_size = train_set[0][0].shape[0]

In [5]:
train_set.__len__()

80

In [33]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# Here we define our model as a class
class LSTM(pl.LightningModule):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2, dnn_shape=1000):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.dnn_shape = dnn_shape

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True)

        # Define the output layer
        self.linear = nn.Linear(self.dnn_shape, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        #print(f"input.view(len(input) {input.view(len(input), self.batch_size, -1).shape}")
        # lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        lstm_out, self.hidden = self.lstm(input)
        #print(lstm_out.shape)
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        # y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        y_pred = self.linear(lstm_out)
        #print(f"y_pred.shape in net: {y_pred.shape}")
        return y_pred


    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.01)
        scheduler = optim.StepLR(optimizer, step_size=100, gamma=0.5)

        return optimizer

    def training_step(self, batch, batch_idx):
        X_batch, y_batch = batch
        #print(f" X_batch:{X_batch.shape}, y_batch: {y_batch.shape}")
        y_pred = self.forward(X_batch)
        #print(f"y_pred.shape: {y_pred.shape}, y_batch.shape(): {y_batch.shape}")
        loss_fn = torch.nn.MSELoss()
        loss = loss_fn(y_pred, y_batch.float())
        # criterion = nn.BCEWithLogitsLoss()
        # loss = criterion(y_pred, y_batch.unsqueeze(1))
        tensorboard_logs = {'train_loss': loss}
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return {'loss': loss}
        


    
    def validation_step(self, batch):
        X_batch, y_batch = batch
        y_pred = self.forward(X_batch)
        
        return {'loss': loss}

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.01)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=800, gamma=0.5)

        return [optimizer], [scheduler]

model = LSTM(1600, 1000, batch_size=20, output_dim=136, num_layers=1)

In [34]:

if True:
    logger = pl.loggers.TensorBoardLogger("training/logs")


    trainer = pl.Trainer(logger=logger, weights_save_path="training/logs", gpus=1 )

    trainer.fit(model, train_loader)
    # trainer.test(model, datamodule=data)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params
----------------------------------
0 | lstm   | LSTM   | 10.4 M
1 | linear | Linear | 136 K 
----------------------------------
10.5 M    Trainable params
0         Non-trainable params
10.5 M    Total params
42.177    Total estimated model params size (MB)
Epoch 999: 100%|██████████| 80/80 [00:06<00:00, 12.50it/s, loss=0.0028, v_num=68, train_loss_step=0.00162, train_loss_epoch=0.00302]


In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# Here we define our model as a class
class LSTM(pl.LightningModule):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2, dnn_shape=1000):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.dnn_shape = dnn_shape

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True)

        # Define the output layer
        self.linear = nn.Linear(self.dnn_shape, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        #print(f"input.view(len(input) {input.view(len(input), self.batch_size, -1).shape}")
        # lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        lstm_out, self.hidden = self.lstm(input)
        #print(lstm_out.shape)
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        # y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        y_pred = self.linear(lstm_out)
        #print(f"y_pred.shape in net: {y_pred.shape}")
        return y_pred


    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        scheduler = optim.StepLR(optimizer, step_size=100, gamma=0.5)

        return optimizer

    def training_step(self, batch, batch_idx):
        X_batch, y_batch = batch
        #print(f" X_batch:{X_batch.shape}, y_batch: {y_batch.shape}")
        y_pred = self.forward(X_batch)
        #print(f"y_pred.shape: {y_pred.shape}, y_batch.shape(): {y_batch.shape}")
        loss_fn = torch.nn.MSELoss()
        loss = loss_fn(y_pred, y_batch.float())
        # criterion = nn.BCEWithLogitsLoss()
        # loss = criterion(y_pred, y_batch.unsqueeze(1))
        tensorboard_logs = {'train_loss': loss}
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return {'loss': loss}
        


    
    def validation_step(self, batch):
        X_batch, y_batch = batch
        y_pred = self.forward(X_batch)
        
        return {'loss': loss}

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.01)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=800, gamma=0.5)

        return [optimizer], [scheduler]



In [ ]:
model = LSTM(1600, 100, batch_size=80, output_dim=136, num_layers=1, dnn_shape=100)
if True:
    logger = pl.loggers.TensorBoardLogger("training/logs")


    trainer = pl.Trainer(logger=logger, weights_save_path="training/logs", gpus=1 )

    trainer.fit(model, train_loader)
    # trainer.test(model, datamodule=data)

In [10]:
model = LSTM.load_from_checkpoint("training/logs/default/version_71/checkpoints/epoch=999-step=79999.ckpt",  input_dim=1600, hidden_dim=100, batch_size=80, output_dim=136, num_layers=1, dnn_shape=100)
# C:\studies\wav2kp\training\logs\default\version_71\checkpoints\epoch=999-step=79999.ckpt


In [170]:
def rescale_keypoints(keypoints, max_size=(256, 256)):
    # multiply all x and y coordinates by the corresponding axis max
    # this is defined on the image processing stage and should be the
    # inverse of the scale_keypoints function
    for p in range(0, 136, 2):
        keypoints[:, :, p] = keypoints[:, :, p] * max_size[0]
        keypoints[:, :, p+1] = keypoints[:, :, p+1] * max_size[1]
    
    
    return keypoints


In [214]:
inf_train = model(train_mfccs)
inf_train = rescale_keypoints(inf_train)
inf_val = model(val_mfccs)
inf_val = rescale_keypoints(inf_val)
inf_test = model(test_mfccs)
inf_test = rescale_keypoints(inf_test)

In [215]:
from skimage import io
from PIL import Image, ImageDraw


In [216]:
inf_train[0,0,10].item()

86.52184295654297

In [213]:
file_path = os.path.abspath("") + "/docs/train_val_test_dist.xlsx"
train_test_dist = pd.read_excel(file_path)

for inf_set, set_name in zip([inf_train, inf_val, inf_test], ["Train", "Val", "Test"]):
    
    file_names = sorted(train_test_dist[train_test_dist.set == set_name].video.unique())    

    for inst in inf_train:
        for sample in inst:
            print(sample.shape)
        
    


torch.Size([136])


In [199]:
def draw_keypoints(frames, file_name, max_size=(256, 256)):
    
    for idx, frame in zip(range(0, len(frames)), frames):
        # print(frame.shape)
        img_d = Image.new("RGB", (max_size[0], max_size[1]) )

        draw = ImageDraw.Draw(img_d)
        points = np.empty([68, 2], dtype=int)
        # frame = frame.detach().numpy()
        for p in range(0, 136, 2):
            current_coord = int(p/2)
            points[current_coord, 0] = frame[p]
            points[current_coord, 1] = frame[p+1]
            draw.ellipse((points[current_coord, 0], points[current_coord, 1], points[current_coord, 0] + 1, points[current_coord, 1] + 1), fill='white', outline='white')


        img_d.save(f"{file_name}/{idx:05}.png")
        
frames = inf_train[0,:].detach()
draw_keypoints(frames, "results")

In [209]:
from utils.v2v_utils import *
def draw_face_edges_v2v(frames, file_name):
    
    part_list = [[list(range(0, 17)) + list(range(68, 83)) + [0]],  # face
                 [range(17, 22)],  # right eyebrow
                 [range(22, 27)],  # left eyebrow
                 [[28, 31], range(31, 36), [35, 28]],  # nose
                 [[36, 37, 38, 39], [39, 40, 41, 36]],  # right eye
                 [[42, 43, 44, 45], [45, 46, 47, 42]],  # left eye
                 [range(48, 55), [54, 55, 56, 57, 58, 59, 48]],  # mouth
                 [range(60, 65), [64, 65, 66, 67, 60]]  # tongue
                 ]

    w, h = 256, 256
    edge_len = 3  # interpolate 3 keypoints to form a curve when drawing edges
    # edge map for face region from keypoints
    # im_edges = np.zeros((h, w), np.uint8) # edge map for all edges


    color = 0
    color_edge = (255, 255, 255)

    for idx, frame in zip(range(0, len(frames)), frames):

        im_edges = np.full((h, w), color, np.uint8)
        keypoints = np.empty([68, 2], dtype=int)
        # frame = frame.detach().numpy()
        for p in range(0, 136, 2):
            current_coord = int(p/2)
            keypoints[current_coord, 0] = frame[p]
            keypoints[current_coord, 1] = frame[p+1]
        print(keypoints.shape)
    
        # from v2v
        pts = keypoints[:17, :].astype(np.int32)
        baseline_y = (pts[0, 1] + pts[-1, 1]) / 2
        upper_pts = pts[1:-1, :].copy()
        upper_pts[:, 1] = baseline_y + (baseline_y - upper_pts[:, 1]) * 2 // 3
        keypoints = np.vstack((keypoints, upper_pts[::-1, :]))

        for edge_list in part_list:
            for edge in edge_list:

                # im_edge = np.full((h, w), color, np.uint8)
                im_edge = np.zeros((h, w), np.uint8)  # edge map for the current edge
                for i in range(0, max(1, len(edge) - 1),
                            edge_len - 1):  # divide a long edge into multiple small edges when drawing
                    sub_edge = edge[i:i + edge_len]
                    x = keypoints[sub_edge, 0]
                    y = keypoints[sub_edge, 1]

                    curve_x, curve_y = interpPoints(x, y)  # interp keypoints to get the curve shape

                    drawEdge(im_edges, curve_x, curve_y, color=color_edge)

        # np.save(im_edges,A_path.replace("txt", "png").replace("keypoints", "gambi_image"))
        im_edges = Image.fromarray(im_edges)
        # im_edges = resize_img(im_edges, keypoints)
        im_edges.save(f"{file_name}/{idx:05}.png")
        # Image.fromarray(crop(im_edges, keypoints)).save(A_path.replace("txt", "png").replace("keypoints", "gambi_image"))
    return

draw_face_edges_v2v(frames, "results/v2v")

(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)
(68, 2)


In [123]:
np.array(range(1, 137))

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136])

In [131]:
orig = np.array(list(zip(list(range(1, 69)), list(range(1, 69)))))
orig

array([[ 1,  1],
       [ 2,  2],
       [ 3,  3],
       [ 4,  4],
       [ 5,  5],
       [ 6,  6],
       [ 7,  7],
       [ 8,  8],
       [ 9,  9],
       [10, 10],
       [11, 11],
       [12, 12],
       [13, 13],
       [14, 14],
       [15, 15],
       [16, 16],
       [17, 17],
       [18, 18],
       [19, 19],
       [20, 20],
       [21, 21],
       [22, 22],
       [23, 23],
       [24, 24],
       [25, 25],
       [26, 26],
       [27, 27],
       [28, 28],
       [29, 29],
       [30, 30],
       [31, 31],
       [32, 32],
       [33, 33],
       [34, 34],
       [35, 35],
       [36, 36],
       [37, 37],
       [38, 38],
       [39, 39],
       [40, 40],
       [41, 41],
       [42, 42],
       [43, 43],
       [44, 44],
       [45, 45],
       [46, 46],
       [47, 47],
       [48, 48],
       [49, 49],
       [50, 50],
       [51, 51],
       [52, 52],
       [53, 53],
       [54, 54],
       [55, 55],
       [56, 56],
       [57, 57],
       [58, 58],
       [59, 59

In [135]:
transp = np.reshape(orig, (1, 136))
transp

array([[ 1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,
         9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16,
        17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24,
        25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32,
        33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40,
        41, 41, 42, 42, 43, 43, 44, 44, 45, 45, 46, 46, 47, 47, 48, 48,
        49, 49, 50, 50, 51, 51, 52, 52, 53, 53, 54, 54, 55, 55, 56, 56,
        57, 57, 58, 58, 59, 59, 60, 60, 61, 61, 62, 62, 63, 63, 64, 64,
        65, 65, 66, 66, 67, 67, 68, 68]])